<a href="https://colab.research.google.com/github/Svein-Tore/colab/blob/main/pi_regulator_niva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installer ved behov:
# !pip install control matplotlib numpy ipywidgets

import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
from ipywidgets import interact, FloatSlider

# =========================
# SIMC PI-regulator
# =========================
def simc_pi(Kp_process, T, L, lam):
    Kc = T/ (Kp_process * (lam + L))
    Ti = T
    s = ctrl.TransferFunction.s
    C = Kc * (1 + 1/(Ti*s))
    return C

# =========================
# FOPDT-prosess
# =========================
def fopdt_system(K, T, L):
    num_pade, den_pade = ctrl.pade(L, 1)
    dead_time = ctrl.TransferFunction(num_pade, den_pade)
    G_no_delay = ctrl.TransferFunction([K], [T, 1])
    return G_no_delay * dead_time

# =========================
# Plot-funksjon
# =========================
def plot_step_response(K=1.0, T=8.0, L=1.0, A=2.0, lam=10.0):

    G = fopdt_system(K, T, L)
    C = simc_pi(K, T, L, lam)

    # Lukket sløyfe
    T_closed = ctrl.feedback(C * G, 1)

    t_end = 5 * (lam + L)
    t = np.linspace(0, t_end, 1000)

    # A er stegamplitude
    t, y = ctrl.step_response(T_closed * A, t)

    plt.figure(figsize=(10, 5))
    plt.plot(t, y)
    plt.title("Steprespons – SIMC PI")
    plt.xlabel("Tid (s)")
    plt.ylabel("y(t)")
    plt.grid(True)
    plt.show()

# =========================
# Sliders
# =========================
interact(
    plot_step_response,
    K=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='K'),
    T=FloatSlider(value=8.0, min=0.1, max=100.0, step=1.0, description='T'),
    L=FloatSlider(value=1.0, min=0.1, max=20.0, step=0.1, description='L'),
    A=FloatSlider(value=2.0, min=0.1, max=100.0, step=0.1, description='A'),
    lam=FloatSlider(value=10.0, min=0.1, max=100.0, step=1.0, description='λ')
)